In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [2]:
c_list = [[83.16, 122.22, 82.62, 133.50, 81.81, 65.70, 52.89],
          [97.20, 98.28, 75.48, 147.00, 40.50, 70.20, 110.94],
          [123.84, 99.12, 76.16, 130.00, 79.92, 38.40, 101.48],
          [75.60, 123.48, 42.84, 195.00, 72.90, 88.20, 59.34],
          [139.32, 107.10, 78.54, 121.50, 36.45, 65.70, 99.33],
          [83.88, 92.82, 124.10, 134.50, 75.87, 69.90, 48.59]]
c= pd.DataFrame(c_list, index=range(1,7), columns=range(1,8))

In [3]:
c #i: potential sites, j: farms

,1,2,3,4,5,6,7
1,83.16,122.22,82.62,133.5,81.81,65.7,52.89
2,97.20,98.28,75.48,147.0,40.50,70.2,110.94
3,123.84,99.12,76.16,130.0,79.92,38.4,101.48
4,75.60,123.48,42.84,195.0,72.90,88.2,59.34
5,139.32,107.10,78.54,121.5,36.45,65.7,99.33
6,83.88,92.82,124.10,134.5,75.87,69.9,48.59


In [4]:
d= pd.Series([36, 42, 34, 50, 27, 30, 43], index=range(1,8))
d# Demands of farms

1    36
2    42
3    34
4    50
5    27
6    30
7    43
dtype: int64

In [5]:
q= pd.Series([80,90, 110, 120, 100, 120], index= range(1,7))
q # Daily forage throughput of potential sites (supplies)

1     80
2     90
3    110
4    120
5    100
6    120
dtype: int64

In [6]:
f= pd.Series([220, 240, 260, 275, 240, 230], index= range(1,7))
f #daily fixed cost of potential sites                 

1    220
2    240
3    260
4    275
5    240
6    230
dtype: int64

In [7]:
def calc_sub_lb(i, lambdas):
    x_i = pd.Series([0] * 7, index=range(1,8), dtype='float64')
    c_i = c.loc[i]
    remaining_cap = q[i]
    
    ratio = (c_i + lambdas) / d
    asc_order = ratio.sort_values().index
    for j in asc_order:
        d_j = d[j]
        x_curr = min(1, remaining_cap/d_j)
        x_i[j] = x_curr
        remaining_cap -= x_curr * d_j
        
        if remaining_cap == 0:
            break
            
    cost = np.sum(x_i * (c_i + lambdas)) + f[i]
    if cost < 0:
        return cost, x_i
    else:
        x_i = pd.Series([0] * 7, index=range(1,8), dtype='float64')
        return 0, x_i

In [8]:
calc_sub_lb(1, [-186, -170,-140,60,-115,-166,-112])

(-6.767058823529396,
 1    1.000000
 2    0.000000
 3    0.411765
 4    0.000000
 5    0.000000
 6    1.000000
 7    0.000000
 dtype: float64)

In [9]:
def calc_lb(lambdas):
    sub_lb_list = []
    x_df = pd.DataFrame()
    for i in range(1,7):
        sub_lb, x_i = calc_sub_lb(i, lambdas)
        sub_lb_list.append(sub_lb)
        x_df = pd.concat([x_df, x_i], axis=1)
        
    x_df = x_df.transpose()
    x_df.index = range(1,7)
    return sum(sub_lb_list) - sum(lambdas), x_df

In [10]:
calc_lb([-186,-170,-140,60,-115,-166,-112])

(681.6271801330352,
           1         2         3    4         5    6        7
 1  1.000000  0.000000  0.411765  0.0  0.000000  1.0  0.00000
 2  0.916667  0.000000  0.000000  0.0  1.000000  1.0  0.00000
 3  1.000000  0.238095  1.000000  0.0  0.000000  1.0  0.00000
 4  1.000000  0.000000  1.000000  0.0  0.740741  1.0  0.00000
 5  0.000000  0.214286  1.000000  0.0  1.000000  1.0  0.00000
 6  1.000000  1.000000  0.000000  0.0  0.000000  1.0  0.27907)

In [11]:
def simplex(facilities):
    coefs = c.loc[facilities, :].to_numpy().flatten()
    num_i = len(facilities)
    
    smaller_coef = np.zeros((num_i,num_i*7), dtype='float64')
    for i in range(num_i):
        j = 7 * i
        smaller_coef[i][j : j + 7] = d
        
    smaller_rhs = q[facilities]
    
    equal_coef = np.zeros((7, num_i * 7))
    for i in range(7):
        equal_coef[i][i::7] = [1] * num_i
    
    equal_rhs = [1] * 7
    
    res = linprog(coefs,
                  A_ub=smaller_coef, b_ub=smaller_rhs, 
                  A_eq=equal_coef, b_eq=equal_rhs, 
                  method='revised simplex')
    return res

In [12]:
def calc_ub(lambdas):
    # Finding facilities to be activated
    activated_fac = []
    demand_satisfied = 0
    total_demand = np.sum(d)
    
    sub_lb_series = pd.Series([calc_sub_lb(i, lambdas)[0] for i in range(1,7)], index=range(1,7))
    asc_order = sub_lb_series.sort_values().index
    
    for i in asc_order:
        if demand_satisfied < total_demand:
            activated_fac.append(i)
            demand_satisfied += q[i]
        else:
            break
    
    # Customer Allocation to the Selected Facilities
    activated_fac.sort()
    
    simplex_result = simplex(activated_fac)
    f_value = simplex_result.fun + np.sum(f[activated_fac])
    x = simplex_result.x
    x_split = np.split(x, len(activated_fac))
    x_df = pd.DataFrame(x_split, columns=range(1,8), index= activated_fac)
    
    return f_value , x_df

In [13]:
calc_ub([-186, -170, -140, 60, -115, -166, -112])

(1229.48,
               1    2    3    4    5             6    7
 4  1.000000e+00  0.0  1.0  0.0  0.0 -2.775558e-17  0.0
 5  0.000000e+00  0.0  0.0  1.0  1.0  7.666667e-01  0.0
 6  1.232595e-32  1.0  0.0  0.0  0.0  2.333333e-01  1.0)

In [14]:
def subgrad(x_df):
    return (x_df.sum(axis=0) - 1).to_numpy()

In [15]:
def B(alpha, ub, lb_h, subgrad_list):
    return (alpha * (ub - lb_h)) / np.sum(subgrad_list**2)
    # En iyi Lower bound ver

In [16]:
def lagrangian(tolerance, alpha, lambdas):
    # Initialization
    lb = np.NINF
    ub = np.Inf
    h = 0
    # New LB and UB, lamba updates
    while True:
        h += 1
        cur_lb, x_df = calc_lb(lambdas)
        cur_ub = calc_ub(lambdas)[0]

        lb = max(lb, cur_lb)
        ub = min(ub, cur_ub)
        norm_dif = (ub-lb)/lb if lb !=0 else np.Inf
        if norm_dif <= tolerance:
            break;
            
        s_j= subgrad(x_df)
        beta = B(alpha, ub, cur_lb, s_j)

        lambdas = lambdas + beta * s_j
    
    return lb, ub, lambdas, beta, s_j, x_df, norm_dif, h

In [17]:
def test_lag(lambdas, lb, ub):

    # New LB and UB, lamba updates
    cur_lb, x_df = calc_lb(lambdas)
    cur_ub = calc_ub(lambdas)[0]

    lb = max(lb, cur_lb)
    ub = min(ub, cur_ub)

    s_j= subgrad(x_df)
    beta = B(0.5, ub, cur_lb, s_j)

    lambdas = lambdas + beta * s_j
    
    return lb, ub, lambdas, beta, s_j

In [18]:
lagrangian(0.1291, 0.5, [0,0,0,0,0,0,0])

(1079.6102950857548,
 1218.08,
 array([-155.88810555, -183.65992848, -146.95856951, -233.23938685,
        -117.55532837, -124.59444568, -138.38367807]),
 6.640789868037714,
 array([ 1.        ,  0.        ,  2.        ,  0.1       ,  0.        ,
         3.        , -0.25581395]),
           1    2    3    4         5    6    7
 1  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0
 2  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0
 3  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0
 4  1.000000  0.0  1.0  0.0  0.259259  0.0  1.0
 5  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0
 6  0.972222  1.0  0.0  0.0  0.000000  0.0  1.0,
 0.12825897043085002,
 415)

In [19]:
lagrangian(0.1291, 0.5, [0,0,0,0,0,0,0])

(1079.6102950857548,
 1218.08,
 array([-155.88810555, -183.65992848, -146.95856951, -233.23938685,
        -117.55532837, -124.59444568, -138.38367807]),
 6.640789868037714,
 array([ 1.        ,  0.        ,  2.        ,  0.1       ,  0.        ,
         3.        , -0.25581395]),
           1    2    3    4         5    6    7
 1  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0
 2  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0
 3  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0
 4  1.000000  0.0  1.0  0.0  0.259259  0.0  1.0
 5  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0
 6  0.972222  1.0  0.0  0.0  0.000000  0.0  1.0,
 0.12825897043085002,
 415)